### 데이터 불러오기 & 피클 만들기 & 호출

In [1]:
import pandas as pd

In [2]:

# df=pd.read_csv("자동차등록.csv",encoding='utf-8')
# df.to_pickle("자동차등록.pkl")


In [ ]:
df = pd.read_pickle("../data/자동차등록.pkl")

### 엑셀로 1차 처리 후 데이터 EDA

In [ ]:
df['항목']

0         승용_관용
1         승용_관용
2         승용_관용
3         승용_관용
4         승용_관용
          ...  
194035     특수_계
194036     특수_계
194037     특수_계
194038     특수_계
194039     특수_계
Name: 항목, Length: 194040, dtype: object

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194040 entries, 0 to 194039
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   시도명     194040 non-null  object        
 1   시군구     194040 non-null  object        
 2   년도      194040 non-null  int64         
 3   월       194040 non-null  int64         
 4   항목      194040 non-null  object        
 5   대수      194040 non-null  int64         
 6   차종      194040 non-null  object        
 7   구분      194040 non-null  object        
 8   date    194040 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 13.3+ MB


In [ ]:
df.isnull().sum()

시도명     0
시군구     0
년도      0
월       0
항목      0
대수      0
차종      0
구분      0
date    0
dtype: int64

### 전처리

날짜 년도, 월로 수정

In [ ]:
df[['년도', '월']] = df['날짜'].str.split('-', expand=True)

df['년도'] = df['년도'].astype(int)
df['월'] = df['월'].astype(int)
df.drop(columns='날짜', inplace=True)

집계를 위해 컬럼 _ 로 나눈거 쪼개기

In [ ]:

def to_long(df):
    id_cols = ['시도명', '시군구', '년도', '월']
    value_cols = [c for c in df.columns if c not in id_cols]

    df_long = df.melt(id_vars=id_cols, value_vars=value_cols, var_name='항목', value_name='대수')
    df_long['항목'] = df_long['항목'].astype(str).str.replace(r'\.\d+$', '', regex=True)

    parts = df_long['항목'].str.split('_', n=1, expand=True)
    df_long['차종'] = parts[0]
    df_long['구분'] = parts[1]
    df_long['date'] = pd.to_datetime(df_long['년도'].astype(str) + '-' + df_long['월'].astype(str).str.zfill(2))
    return df_long


In [ ]:
df=to_long(df)

### 전처리 후 다시 EDA

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194040 entries, 0 to 194039
Data columns (total 9 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   시도명     194040 non-null  object        
 1   시군구     194040 non-null  object        
 2   년도      194040 non-null  int64         
 3   월       194040 non-null  int64         
 4   항목      194040 non-null  object        
 5   대수      194040 non-null  int64         
 6   차종      194040 non-null  object        
 7   구분      194040 non-null  object        
 8   date    194040 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 13.3+ MB


In [11]:
df.shape

(194040, 9)

In [ ]:
df.head()

,시도명,시군구,년도,월,항목,대수,차종,구분,date
0,서울,강남구,2022,1,승용_관용,151,승용,관용,2022-01-01
1,서울,강동구,2022,1,승용_관용,103,승용,관용,2022-01-01
2,서울,강북구,2022,1,승용_관용,104,승용,관용,2022-01-01
3,서울,강서구,2022,1,승용_관용,127,승용,관용,2022-01-01
4,서울,관악구,2022,1,승용_관용,94,승용,관용,2022-01-01


In [15]:
df.to_pickle("자동차등록.pkl")

### Plotly 기반 시각화

In [5]:
df=pd.read_pickle("data\자동차등록.pkl")

In [ ]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output


# -------------------------
# 2) 위젯 구성
# -------------------------
sido_options = sorted(df['시도명'].unique())
sido_dd = widgets.Dropdown(options=sido_options, description='시도명:', value=sido_options[0])

# 시도에 따라 시군구 옵션이 달라지게
def get_sigungu_options(sido):
    return sorted(df.loc[df['시도명'] == sido, '시군구'].unique())

sigungu_dd = widgets.Dropdown(options=get_sigungu_options(sido_options[0]), description='시군구:')

car_options = sorted(df['차종'].unique())
car_dd = widgets.Dropdown(options=car_options, description='차종:', value=car_options[0])

gubun_options = sorted(df['구분'].unique())
gubun_dd = widgets.Dropdown(options=gubun_options, description='구분:', value=gubun_options[0])

agg_dd = widgets.Dropdown(
    options=[('월별(라인)', 'line'), ('월별(막대)', 'bar')],
    description='차트:'
)

out = widgets.Output()

# 시도 바뀌면 시군구 옵션 갱신
def on_sido_change(change):
    new_sido = change['new']
    sigungu_dd.options = get_sigungu_options(new_sido)
    sigungu_dd.value = sigungu_dd.options[0] if sigungu_dd.options else None

sido_dd.observe(on_sido_change, names='value')

# -------------------------
# 3) 렌더 함수
# -------------------------
def render(*args):
    with out:
        clear_output(wait=True)

        sido = sido_dd.value
        sigungu = sigungu_dd.value
        car = car_dd.value
        gubun = gubun_dd.value
        chart_type = agg_dd.value

        dff = df[
            (df['시도명'] == sido) &
            (df['시군구'] == sigungu) &
            (df['차종'] == car) &
            (df['구분'] == gubun)
        ].sort_values('date')

        title = f"{sido} {sigungu} | {car} - {gubun} (월별)"

        if chart_type == 'line':
            fig = px.line(
                dff, x='date', y='대수',
                markers=True,
                title=title
            )
        else:
            fig = px.bar(
                dff, x='date', y='대수',
                title=title
            )

        fig.update_layout(
            xaxis_title="월",
            yaxis_title="등록 대수",
            hovermode="x unified"
        )
        fig.update_xaxes(dtick="M1", tickformat="%Y-%m", tickangle=-45)
        fig.show()

# 위젯 변경 시 자동 갱신
for w in [sido_dd, sigungu_dd, car_dd, gubun_dd, agg_dd]:
    w.observe(render, names='value')

display(widgets.VBox([widgets.HBox([sido_dd, sigungu_dd]), widgets.HBox([car_dd, gubun_dd, agg_dd]), out]))
render()


In [4]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# df는 이미 존재한다고 가정 (columns: 시도명, 시군구, 년도, 월, 항목, 대수, 차종, 구분, date)

def filter_df(df, sido=None, sigungu=None, car=None, gubun=None, date_from=None, date_to=None):
    dff = df.copy()
    if sido is not None:
        dff = dff[dff["시도명"].isin([sido] if isinstance(sido, str) else sido)]
    if sigungu is not None:
        dff = dff[dff["시군구"].isin([sigungu] if isinstance(sigungu, str) else sigungu)]
    if car is not None:
        dff = dff[dff["차종"].isin([car] if isinstance(car, str) else car)]
    if gubun is not None:
        dff = dff[dff["구분"].isin([gubun] if isinstance(gubun, str) else gubun)]
    if date_from is not None:
        dff = dff[dff["date"] >= pd.to_datetime(date_from)]
    if date_to is not None:
        dff = dff[dff["date"] <= pd.to_datetime(date_to)]
    return dff

# 월 문자열 컬럼(카테고리 축/라벨링용)
df = df.copy()
df["month"] = df["date"].dt.strftime("%Y-%m")


In [5]:
import plotly.express as px

car_list = df["차종"].unique()
gubun_list = df["구분"].unique()

# 초기값
car0 = car_list[0]
gubun0 = gubun_list[0]

dff = filter_df(df, car=car0, gubun=gubun0)

agg = (
    dff.groupby(["month", "시도명"], as_index=False)["대수"].sum()
    .sort_values(["month", "대수"], ascending=[True, False])
)

fig = px.bar(
    agg,
    x="대수",
    y="시도명",
    orientation="h",
    animation_frame="month",
    title=f"시도별 등록대수 ({car0}-{gubun0})"
)

# ---------------- 버튼 ----------------
buttons = []

for car in car_list:
    for gubun in gubun_list:
        dff_btn = filter_df(df, car=car, gubun=gubun)

        agg_btn = (
            dff_btn.groupby(["month", "시도명"], as_index=False)["대수"].sum()
            .sort_values(["month", "대수"], ascending=[True, False])
        )

        buttons.append(
            dict(
                label=f"{car}-{gubun}",
                method="update",
                args=[
                    {
                        "x": [agg_btn["대수"]],
                        "y": [agg_btn["시도명"]],
                    },
                    {
                        "title": f"시도별 등록대수 ({car}-{gubun})"
                    }
                ],
            )
        )

fig.update_layout(
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            x=0.02,
            y=1.15,
        )
    ],
    yaxis={"categoryorder": "total ascending"},
)

fig.show()


In [ ]:
car = "승용"      # 원하는 차종
gubun = "계"      # 예: "자가용", "영업용", "관용", "계" 등 실제 값으로 맞춰줘

dff = filter_df(df, car=car, gubun=gubun)

agg = (dff.groupby(["month", "시도명"], as_index=False)["대수"].sum()
       .sort_values(["month", "대수"], ascending=[True, False]))

fig = px.bar(
    agg, x="대수", y="시도명", orientation="h",
    animation_frame="month",
    title=f"년도 등록대수 변화 ({car}-{gubun})",
)
fig.update_layout(yaxis={"categoryorder":"total ascending"})
fig.show()
